In [1]:
import sys

sys.path.append("../src")
from tokenizer import get_tokenizer, get_data
from torch.utils.data import Dataset, DataLoader
from CustomDataLoader import CustomDataset


dataset_path = (
    "/Users/shusanketbasyal/.cache/kagglehub/datasets/jigarpanjiyar/english-to-manipuri-dataset/versions/1"
    + "//english-nepali.xlsx"
)

engtokenizer, neptokenizer = get_tokenizer(dataset_path)

df_train, df_test = get_data(dataset_path, split=True)

engvocabsize = engtokenizer.get_vocab_size()
nepvocabsize = neptokenizer.get_vocab_size()

df_train_dataset = CustomDataset(
    df_train, engtokenizer, neptokenizer, "eng", "nep", 256
)
df_test_dataset = CustomDataset(df_test, engtokenizer, neptokenizer, "eng", "nep", 256)

df_train_dataloader = DataLoader(df_train_dataset, batch_size=1, shuffle=True)

In [2]:
for x in df_train_dataloader:
    enc_input = x["encoder_input"]
    dec_input = x["decoder_input"]
    enc_mask = x["encoder_mask"]
    dec_mask = x["decoder_mask"]

    print(enc_input.shape)
    print(dec_input.shape)
    print(x["label"].shape)
    print(enc_mask.shape)
    print(dec_mask.shape)

    break

torch.Size([1, 256])
torch.Size([1, 256])
torch.Size([1, 256])
torch.Size([1, 256])
torch.Size([1, 256, 256])


In [7]:
from model import build_transformer
import torch
import torch.nn as nn

transformer = 1

transformer = build_transformer(
    engtokenizer.get_vocab_size(), 256, neptokenizer.get_vocab_size(), 256, 512, 4, 4, 8
)

In [4]:
s = 0
for p in transformer.parameters():
    s += p.numel()
print(s)

75797808


In [5]:
# training loop
device = torch.device()
for x in df_train_dataloader:
    enc_input = x["encoder_input"]
    dec_input = x["decoder_input"]
    enc_mask = x["encoder_mask"]
    dec_mask = x["decoder_mask"]
    label = x["label"]

    encoder_output = transformer.encoder_fun(enc_input, enc_mask)

    decoder_output = transformer.decoder_fun(
        dec_input, encoder_output, enc_mask, dec_mask
    )
    logits = transformer.projection(decoder_output)

    break

In [6]:
torch

NameError: name 'torch' is not defined